# Representação de documentos e cálculo de similaridade

In [11]:
import sys
import gzip
import numpy as np
import glob
import collections
import operator
import nltk

Caso esteja a utilizar o google colab, antes de continuar certifique-se que fez o download dos dados previamente conforme instruções fornecidas em [download dos dados utilizados nos exemplos](./using-tm-data.ipynb)

In [ ]:

if 'google.colab' in str(get_ipython()):
    nltk.download('punkt')
    from google.colab import drive
    drive.mount('/content/drive')
    %cd /content/drive/MyDrive/Iscte/TM/data

In [ ]:
ficheiros=glob.glob("../data/pt/tsf.selecionado/*.gz")
print(ficheiros)

In [ ]:
textos = []
for fname in ficheiros:
    f=gzip.open(fname, "rt", encoding="utf-8")
    texto = f.readlines()
    texto = "".join(texto)
    textos.append(texto)
print("Tenho guardados %d documentos"% len(textos))

In [ ]:
print(textos[1])

Agora vamos extrair contagens de palavras (features) a partir dos documentos.

Nesta fase talvez não fosse má ideia fazer algum processamento, por exemplo, para passar a ter apenas palavras em letras minúsculas e evitar ter palavras no vocabulário tais como `«perfeito»` ou `«já`...

In [ ]:
def doc2wc(texto):   
    palavras = nltk.word_tokenize(texto.lower())
    return collections.Counter(palavras)

doc2wc("A capa do livro do Pedro")

In [ ]:
docs = []
for texto in textos:
    contagens = doc2wc(texto)
    docs.append(contagens)
print(docs[0])

Talvez não fosse má ideia fazer algum processamento para, por exemplo:
 * passar a ter apenas palavras em letras minúsculas
 * eliminar tokens tais como `(`, `:`, `«`, etc...
 * evitar ter palavras no vocabulário tais como `«perfeito»` ou `«já`.

Vamos agora ver qual é o vocabulário que está a ser usado ...

In [115]:
voc = set()
for d in docs:
    for w in d:
        voc.add(w)

In [ ]:
print("O vocabulário tem %d palavras"% len(voc) )
print(voc)

### (1) Vamos representar os documentos usando apenas as palavras do vocabulário

Vamos agora criar um identificador para cada palavra do vocabulário

In [117]:
def create_index(voc):
    voc_ids = {}
    for i,w in enumerate(sorted(voc)):
        voc_ids[w] = i
    return voc_ids

In [ ]:
voc_ids = create_index(voc)
print(voc_ids)

Agora sim, vamos representar os nossos documentos em vetores numéricos

In [ ]:
def doc2vec(doc, voc_ids):
    doc_vector = np.zeros([len(voc_ids)], dtype=int)
    for w in doc:
        if w in voc_ids:
            doc_vector[ voc_ids[w] ] = doc[w]
    return doc_vector

docrep = []
for doc in docs:
    doc_vector = doc2vec(doc, voc_ids)
    docrep.append ( doc_vector )
    
print("número de documentos:", len(docrep))
print("tamanho do documento 0 (é o mesmo para todos os documentos):", len(docrep[0]))
print("documento 0:\n", docrep[0])

Vamos lá ver qual o documento mais próximo do segundo

In [ ]:
from scipy.spatial import distance
ref=1
for i in range(len(docrep)):
    s = 1 - distance.cosine(docrep[ref], docrep[i])
    print(f"similarity({ref},{i}) is {s}")

In [ ]:
print(textos[ref])

In [ ]:
print(textos[5])

## IDF para identificar os termos mais relevantes

In [123]:
def idf(df, ndocs):
    return np.log(ndocs/df)

In [124]:
tf = collections.Counter()
df = collections.Counter()
for d in docs:
    for w in d:
        tf[w] += d[w]
        df[w] += 1
        
idfs = {}
for w in tf:
    if tf[w] > 1:
        idfs[w] = idf(df[w], len(docs))

In [ ]:
for w in ["em", "com", "de", "uefa", "marcador", "voto", "cristiano", "tap"]:
    print(w, tf[w], df[w], len(docs), idf(df[w], len(docs)))

In [ ]:
sorted(idfs.items(), key=operator.itemgetter(1), reverse=True)

Agora que sabemos quais as palavras mais importantes, podemos definir novo vocabulário ...

In [ ]:
voc2 = sorted(idfs, key=idfs.get, reverse=True)[:250]
voc2

Podemos agora voltar a executar as instruções abaixo do ponto (1) e representar os documentos com este novo vocabulário. 
Poderá chegar à conclusão que o par de documentos mais similar não é, afinal, o 3 e o 5.

In [ ]:
voc2_ids = create_index(voc2)
print(voc2_ids)

In [ ]:
docrep2 = []
for doc in docs:
    doc_vector = doc2vec(doc, voc2_ids)
    docrep2.append ( doc_vector )
    
print("número de documentos:", len(docrep2))
print("tamanho do documento 0 (é o mesmo para todos os documentos):", len(docrep2[0]))
print("documento 0:\n", docrep2[0])

In [ ]:
ref=3
for i in range(len(docrep2)):
    s = 1 - distance.cosine(docrep2[ref], docrep2[i])
    print(f"similarity({ref},{i}) is {s}")